# VacationPy
---

In [259]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [260]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,19.19,93,88,1.53,US,1686538683
1,1,vilyuchinsk,52.9306,158.4028,23.15,52,84,1.34,RU,1686538683
2,2,olonkinbyen,70.9221,-8.7187,0.92,98,100,5.20,SJ,1686538683
3,3,svetlyy,54.6750,20.1347,11.65,71,5,3.91,RU,1686538684
4,4,waitangi,-43.9535,-176.5597,12.68,88,100,0.45,NZ,1686538684


### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [261]:
# Configure the map
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    #"Country",
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600,
    color = "City",
    size = "Humidity",
)

# Display the map plot
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [263]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,ain sefra,32.7500,-0.5833,24.29,34,0,3.22,DZ,1686538685
39,39,klyuchi,52.2667,79.1667,21.20,40,0,0.45,RU,1686538693
139,139,ghardaia,32.4909,3.6735,26.14,41,0,3.75,DZ,1686538719
158,158,birjand,32.8663,59.2211,21.26,23,0,3.09,IR,1686538724
190,190,mhamid,29.8200,-5.7200,26.38,20,0,2.14,MA,1686538732
232,232,fort mcmurray,56.7268,-111.3810,25.73,33,0,4.12,CA,1686538743
300,300,rio branco,-9.9747,-67.8100,24.34,100,0,1.03,BR,1686538721
303,303,fort bragg,35.1390,-79.0060,21.78,94,0,2.06,US,1686538762
310,310,vila velha,-20.3297,-40.2925,23.97,90,0,4.12,BR,1686538627
354,354,grande prairie,55.1667,-118.8027,24.10,38,0,1.54,CA,1686538775


### Step 3: Create a new DataFrame called `hotel_df`.

In [268]:
# Create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.iloc[:, [1,8,2,3,5]]

# Add an empty column, "Hotel Name," to the DataFrame to store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "
hotel_df

/var/folders/cz/_cs1cvcn1fz9j44pqtb33vph0000gn/T/ipykernel_65797/3364010178.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = " "


,City,Country,Lat,Lng,Humidity,Hotel Name
8,ain sefra,DZ,32.7500,-0.5833,34,
39,klyuchi,RU,52.2667,79.1667,40,
139,ghardaia,DZ,32.4909,3.6735,41,
158,birjand,IR,32.8663,59.2211,23,
190,mhamid,MA,29.8200,-5.7200,20,
232,fort mcmurray,CA,56.7268,-111.3810,33,
300,rio branco,BR,-9.9747,-67.8100,100,
303,fort bragg,US,35.1390,-79.0060,94,
310,vila velha,BR,-20.3297,-40.2925,90,
354,grande prairie,CA,55.1667,-118.8027,38,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [269]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ain sefra - nearest hotel: فندق مكثر
klyuchi - nearest hotel: No hotel found
ghardaia - nearest hotel: فندق بلعجال
birjand - nearest hotel: مهمانسرای جهانگردی بیرجند
mhamid - nearest hotel: Hotel Kasbah Azalay
fort mcmurray - nearest hotel: Nomad Hotel and Suites
rio branco - nearest hotel: No hotel found
fort bragg - nearest hotel: Airborne Inn Lodging
vila velha - nearest hotel: Hotel Prainha
grande prairie - nearest hotel: Delta Hotels by Marriott Grande Prairie Airport
smithers - nearest hotel: Sunshine Inn Hotel
arraial do cabo - nearest hotel: No hotel found
barakani - nearest hotel: Al Amal
galliano - nearest hotel: Fairfield Inn & Suites by Marriott Cut Off-Galliano
urzhar - nearest hotel: Sharikan


,City,Country,Lat,Lng,Humidity,Hotel Name
8,ain sefra,DZ,32.7500,-0.5833,34,فندق مكثر
39,klyuchi,RU,52.2667,79.1667,40,No hotel found
139,ghardaia,DZ,32.4909,3.6735,41,فندق بلعجال
158,birjand,IR,32.8663,59.2211,23,مهمانسرای جهانگردی بیرجند
190,mhamid,MA,29.8200,-5.7200,20,Hotel Kasbah Azalay
232,fort mcmurray,CA,56.7268,-111.3810,33,Nomad Hotel and Suites
300,rio branco,BR,-9.9747,-67.8100,100,No hotel found
303,fort bragg,US,35.1390,-79.0060,94,Airborne Inn Lodging
310,vila velha,BR,-20.3297,-40.2925,90,Hotel Prainha
354,grande prairie,CA,55.1667,-118.8027,38,Delta Hotels by Marriott Grande Prairie Airport


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [270]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    #"Country",
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600,
    color = "City",
    size = "Humidity",
    hover_cols = ["Hotel Name","Country"]
)

# Display the map plot
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)